In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch import nn
from torch import optim

from tqdm.notebook import tqdm

from nlp.dataset.dataset import TextDataset, collate_fn
from nlp.dataset.datamodule import TextDatamodule
from nlp.model import TextModel
from nlp.pl_module import TextModule

from pytorch_lightning import Trainer
import re

In [2]:
SEQLEN = 3
BATCH_SIZE = 512
MAX_EPOCH = 100
DEVICE = 'cuda:0'
NUM_WORKERS = 4

with open('data/external_text/azattyq.txt', 'r',encoding='utf-8') as file:
    azattyq = file.read().replace('\n', '')
with open('data/external_text/baq.txt', 'r',encoding='utf-8') as file:
    baq = file.read().replace('\n', '')
text = azattyq + ' ' + baq
text = text.replace('.', ' ')
text = re.sub('\\s+', ' ', text)
text = text.split(' ')
word_count = pd.value_counts(text)

tokens = list(word_count[(word_count>30)].keys())
tokens = ['<pad>'] + tokens + ['ukn']
stoi = {token : idx for idx, token in enumerate(tokens)}
itos = tokens

In [3]:
datamodule = TextDatamodule(text, stoi, SEQLEN, BATCH_SIZE, 4)

model = TextModel(len(stoi))
criterion = nn.CrossEntropyLoss()
pl_module = TextModule(model, stoi, itos, criterion, 0.001)
trainer = Trainer(max_epochs=MAX_EPOCH, accelerator='gpu')

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [4]:
trainer.fit(pl_module, datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | TextModel        | 9.4 M 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
9.4 M     Trainable params
0         Non-trainable params
9.4 M     Total params
37.772    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

C:\Users\Abylaikhan\anaconda3\lib\site-packages\pytorch_lightning\trainer\trainer.py:724: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [17]:
pred = pl_module.generate("наука", 19)
result = ''
for i in pred:
    result += i + ' '
result

'ukn ukn ukn ukn ukn ukn ukn ukn ukn ukn ukn ukn ukn ukn ukn ukn ukn ukn ukn '

In [10]:
text

['франция',
 'президенті',
 'эммануэль',
 'макрон',
 'күзде',
 'жылы',
 'қарашада',
 'парижде',
 'болған',
 'терактіден',
 'соң',
 'енгізілген',
 'төтенше',
 'жағдай',
 'режимін',
 'жоятынын',
 'мәлімдеді',
 'франция',
 'парламенті',
 'палаталарының',
 'біріккен',
 'отырысында',
 'елдің',
 'ішкі',
 'және',
 'сыртқы',
 'саясатындағы',
 'басымдықтары',
 'туралы',
 'мәлімдеме',
 'жасаған',
 'макрон',
 'үкімет',
 'экстремизммен',
 'күресу',
 'үшін',
 'қауіпсіздік',
 'шараларын',
 'күшейтетінін',
 'де',
 'айтты',
 'осы',
 'күзде',
 'төтенше',
 'жағдай',
 'режимін',
 'жойып',
 'француз',
 'халқының',
 'еркіндіктерін',
 'қалпына',
 'келтіремін',
 'себебі',
 'бұл',
 'еркіндіктер',
 'күшті',
 'демократияның',
 'алғышарты',
 'саналады',
 'деді',
 'макрон',
 'ол',
 'үкімет',
 'кезкелген',
 'жаңа',
 'шабуыл',
 'жасаудың',
 'алдын',
 'алуға',
 'жұмыс',
 'істейтінін',
 'және',
 'экстремистермен',
 'аяусыз',
 'күресетінін',
 'де',
 'айтты',
 'макрон',
 'бұған',
 'қоса',
 'жеке',
 'адам',
 'еркіндігін